# Backpropagation: Exemplo passo-a-passo



Neste notebook, faremos um exemplo passo a passo do algoritmo backpropagation em python. Foi baseado no post 
[A Step By Step Backpropagation](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/).


Primeiramente vamos começar com os dados iniciais de pesos, bias e dados de treinamento (input/output).

![Exemplo de rede utilizada](img/neural_network.png)

In [9]:
weights = [.15, .20, .25, .30, .40, .45, .50, .55]
bias = [.35, .60]
inputs = [.05, .10]
outputs = [.01, .99]

O objetivo do backpropagation é otimizar o valor dos pesos para que a rede neural possa mapear corretamente os inputs aos outputs.

## The Forward Pass

Primeiramente vamos verificar o que a rede neural com os dados iniciais preve, para isso vamos por os inputs que passará atraves da rede e vai dar um output diferente do esperado, com isso em seguida calcularemos os erros. Relembrando a função net temos:

$ \hat{y} = f\left(\text{net}\right)= f\left(\vec{w}\cdot\vec{x}+b\right) = f\left(\sum_{i=1}^{n}{w_i x_i + b}\right). $

In [10]:
net_h1 = weights[0]*inputs[0] + weights[1]*inputs[1] + bias[0]
net_h1

0.3775

Criaremos em python a função logistica: 
$ \hat{y} = f(\text{net}) = \frac{1}{1 + e^{(-\text{net})}}. $


In [11]:
import numpy as np

In [12]:
def logistic(net):
    return 1.0 / (1.0 + np.exp(-net))

Agora passaremos os dados para ela

In [13]:
out_h1 = logistic(net_h1)
out_h1

0.59326999210718723

Realizando o mesmo processo para out_h2 temos:

In [14]:
net_h2 = weights[2]*inputs[0] + weights[3]*inputs[1] + bias[0]
out_h2 = logistic(net_h2)
out_h2

0.59688437825976703

Agora vamos repetir este processo com as saidas obtidas, assim como mostra a figura acima.

In [15]:
net_o1 = weights[4]*out_h1 + weights[5]*out_h2 + bias[1]
out_o1 = logistic(net_o1)
out_o1

0.75136506955231575

In [16]:
net_o2 = weights[6]*out_h1 + weights[7]*out_h2 + bias[1]
out_o2 = logistic(net_o2)
out_o2

0.77292846532146253

## Calculating the Total Error


Para calcular o erro temos:
$ E_{total} = \frac{1}{2} \sum \left(target - output \right)^2\,. $

In [17]:
def error(target, output):
    return 1.0/2.0 * (target - output)**2

Agora podemos computar as duas saidas

In [18]:
error_o1 = error(out_o1, outputs[0])
error_o1

0.27481108317615499

In [19]:
error_o2 = error(out_o2, outputs[1])
error_o2

0.023560025583847746

In [20]:
error_total = error_o1 + error_o2
error_total

0.29837110876000272

## The Backward Pass

Nosso objetivo é atualizar cada um dos pesos da rede para que os novos inputs estajam o mais proximo possivel dos outputs reais que são informados na fase de treinamento. Minimizando o erro de cada neuronio de saida e da rede como um todo.

![Exemplo de propagacao](img/output_1_backprop-4.png)

Agora vamos desmenbrar cada um dos elementos de 
$ \frac{\partial E_{total}}{\partial w_5} = \frac{\partial E_{total}}{\partial out_{o1}} * \frac{\partial out_{o1}}{\partial net_{o1}} * \frac{\partial net_{o1}}{\partial w_5} $ 
individualmente e depois juntar os elementos.

Para o primeiro elemento do produto temos: 

$ E_{total} = \frac{1}{2} * (target_{o1} - out_{o1})^2 + \frac{1}{2} * (target_{o2} - out_{o2})^2 $

$ \frac{\partial E_{total}}{\partial out_{o1}}  = 2 * \frac{1}{2} * (target_{o1} - out_{o1})^{2-1} * {-1} + 0 $

$ = - (target_{o1} - out_{o1}) $

$ = out_{o1} - target_{o1} $

Para o segundo temos:

$ out_{o1} = \frac{1}{1 + e^{-\text{net}}} $

$ \frac{\partial out_{o1}}{\partial net_{o1}} = -1 * (1 + e^{\text{-net}})^{-1 -1} * e^{-net} * -1 $

$ = \frac{e^{-\text{net}}}{(1 + e^{-\text{net}})^2 } $

$ = \frac{1 + e^{-\text{-net}} -1}{(1 + e^{-\text{net}})^2} $

$ = \frac{1 + e^{-\text{-net}}}{(1 + e^{-\text{net}})^2} - (\frac{1}{1 + e^{-\text{net}}})^2 $

$ = \frac{1}{(1 + e^{-\text{net}})} - (\frac{1}{1 + e^{-\text{net}}})^2 $

$ = out_{o1} - out_{o1}^2 $

$ = out_{o1}*(1 - out_{o1}) $

E por ultimo ...

$ net_{o1} = w_5 * out_{h1} + w_6 * out_{h2} + b_2*1 $

$ \frac{ \partial net_{o1}}{\partial w_5} = 1 * out_{h1} * w_5 ^ {1 - 1} + 0 + 0 = out_{h1} $

Juntando tudo: 

$ \frac{\partial E_{total}}{\partial w_5} = (out_{o1} - target_{o1}) * out_{o1}*(1 - out_{o1}) * out_{h1} $

In [38]:
e_total_w_5 = (out_o1 - outputs[0]) * out_o1 * (1 - out_o1) * out_h1 
e_total_w_5

0.08216704056423077

Atualizando o peso $ w_5 $ utilizamos $ w_5^+ = w_5 + \eta * \frac{\partial E_{total}}{\partial w_5} $

In [39]:
w_5_plus = weights[4] - 0.5 * e_total_w_5
w_5_plus

0.35891647971788465

Repetindo o processo para os outros pesos temos: 

In [33]:
e_total_w_6 = (out_o1 - outputs[0]) * out_o1 * (1 - out_o1) * out_h2 
w_6_plus = weights[5] - 0.5 * e_total_w_6
w_6_plus

0.4086661860762334

In [40]:
e_total_w_7 = (out_o2 - outputs[1]) * out_o2 * (1 - out_o2) * out_h1 
w_7_plus = weights[6] - 0.5 * e_total_w_7
w_7_plus

0.5113012702387375

In [41]:
e_total_w_8 = (out_o2 - outputs[1]) * out_o2 * (1 - out_o2) * out_h2
w_8_plus = weights[7] - 0.5 * e_total_w_8
w_8_plus

0.56137012110798912